In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import csv
from datetime import datetime, timedelta
from selenium.common.exceptions import TimeoutException
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
path = "C:\\Users\\DeepLearning_4\\Desktop\\driver\\chromedriver.exe"

driver = webdriver.Chrome(path , options=options)
driver.set_window_size(360, 740)
driver.get('https://everytime.kr//login')
id = input('id: ')
pw = input('pw: ')
driver.find_element_by_name('userid').send_keys(id)
driver.find_element_by_name('password').send_keys(pw)
driver.find_element_by_class_name('submit').click()
driver.implicitly_wait(1)

In [ ]:
start_date = input('date to start(2020-01-01): ')
s_year, s_month, s_day = map(int, start_date.split('-'))
start_datetime = datetime(s_year, s_month, s_day)

end_date = input('date to end(2020-01-01): ')
e_year, e_month, e_day = map(int, end_date.split('-'))
end_datetime = datetime(e_year, e_month, e_day)

In [ ]:
def make_link():
    page = 1
    links = []

    start_time = time.time()

    is_loop_end = False
    while not is_loop_end:
        driver.get('https://everytime.kr/389148/p/' + str(page))
        try:
            WebDriverWait(driver, 200).until(EC.presence_of_all_elements_located((By.ID, "writeArticleButton")))
        except TimeoutException:
            continue
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')
        # todo time 태그 찾아서 오늘 날짜 이후로 일주일 단위 검색

        linkList = soup.findAll("a", href=re.compile("(\/389148\/v\/........)"))
        # # 마지막 페이지인지 확인
        # if len(linkList) < 20:
        #     break
        #
        # 테스트용
        if page == 101:
            break

        for link in linkList:
            links.append(link.attrs['href'])
        print(page)



        # 한번에 한페이지씩 크롤링하기 위해 추가
        now = datetime(datetime.today().year, datetime.today().month, datetime.today().day)
        for find_time in soup.findAll("time"):
            if '분 전' in find_time.text or '방금' in find_time.text:
                # 오늘 날짜에 올라온 데이터
                post_datetime = now
                now = post_datetime
            elif len(find_time.text) == 11:  
                p_month, p_day = map(int, find_time.text[0:5].split('/'))
                post_datetime = datetime(2021, p_month, p_day)
                now = post_datetime
            else:
                p_year, p_month, p_day = map(int, find_time.text[0:9].split('/'))
                post_datetime = datetime(p_year, p_month, p_day)
                now = post_datetime
                
            if now <= start_datetime:
                if '분 전' in find_time.text or '방금' in find_time.text:
                    # 오늘 날짜에 올라온 데이터
                    post_datetime = now
                    crawling(links.pop())
                else:
                    p_month, p_day = map(int, find_time.text[0:5].split('/'))
                    post_datetime = datetime(2021, p_month, p_day)
                    now = post_datetime
                    crawling(links.pop())

            if post_datetime is not None:
                if post_datetime < end_datetime:
                    is_loop_end = True


        if time.time() - start_time > 3600:
            driver.delete_all_cookies()
            driver.get('https://everytime.kr/login')
            print('session clear')
            driver.find_element_by_name('userid').send_keys(id)
            driver.find_element_by_name('password').send_keys(pw)
            driver.find_element_by_class_name('submit').click()
            start_time = time.time()

        page += 1
        

In [ ]:
def crawling(link):

    
    driver.get('https://everytime.kr' + str(link))
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#container > "
                                                                                              "div.articles > article"
                                                                                              " > a.article")))
    except TimeoutException:
        return
    
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    find_time = soup.find('div', {'class': 'profile'}).find('time').text
    
    # ~분전이라고 표시되는 개시물이 존재함
    if '분' in find_time or '방금' in find_time:
        now = datetime.now()
        title_time = str(now.year) + '/' + str(now.month) + '/' + str(now.day)
    elif len(find_time) == 11:  
        p_month, p_day = find_time[0:5].split('/')
        title_time = '2021' + '/' + p_month + '/' + p_day
    else:
        p_year, p_month, p_day = find_time[0:9].split('/')
        title_time = p_year + '/' + p_month + '/' + p_day
    temp = title_time

    
    title = soup.find('div', {'class': 'wrap articles'}).article.h2.text
    temp = temp + '|' + title
#     writeCSV('title', title, title_time)
    
    content = soup.find('div', {'class': 'wrap articles'}).article.a.p.text
    temp = temp + '|' + content
#     writeCSV('content', content)
    
    commentList = []
    for commentObj in soup.find('div', {'class': 'comments'}).children:
        try:
            comment = commentObj.find('p').text
            commentList.append(comment)       
#            writeCSV('comment', comment, comment_time)
        except AttributeError:
            pass
    commentL = '[' + '|'.join(commentList) + ']'
    temp = temp + '|' + commentL
    writeCSV(temp)
    
    print(link)
    

In [ ]:
def writeCSV(content):
    f = open('에타자유게시판.csv', 'a', encoding='utf-8', newline='')
    wr = csv.writer(f)
    wr.writerow([content])


In [ ]:
make_link()
# df = pd.DataFrame(data, columns=['날짜', 'title', 'content', 'comment'])
# df.to_csv('에타자유게시판2021.csv')  
print('끝')
driver.close()